In [1]:
import os,sys,glob
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats as st
import functions as fun

In [2]:
scens = ['rcp45','rcp85']

In [3]:
#Fingerprints

#Get 1x1 degree grid
with xr.open_dataset('../rawdata/rsl_85_2090.nc') as ds:
    lon = ds['x'].values
    lat = ds['y'].values
    slr_md = ds['slr_md'].values

with xr.open_dataset('../rawdata/fp_uniform/AIS.nc') as ds:
    lon05  = ds['x'].values
    lat05  = ds['y'].values
    fp05_a = ds['rsl'].values

with xr.open_dataset('../rawdata/fp_uniform/GrIS.nc') as ds:
    fp05_gr = ds['rsl'].values
    
#Convert to 1x1 degree grid
fp_a  = np.zeros((len(lat),len(lon)))
fp_gr = np.zeros((len(lat),len(lon)))

for i,ii in enumerate(lat):
    ig = np.argmin((ii-lat05)**2)
    for j,jj in enumerate(lon):
        jg = np.argmin((jj-lon05)**2)
        fp_a[i,j] = fp05_a[ig,jg]
        fp_gr[i,j] = fp05_gr[ig,jg]

In [4]:
#Adyn from script

adyn = np.zeros((2,len(lat),len(lon)))
adyn[0,:,:] = 4.3 * np.abs(fp_a)
adyn[1,:,:] = 6.0 * np.abs(fp_a)
for s in [0,1]:
    adyn[s,:,:][np.isnan(slr_md)]=np.nan

In [5]:
#Gsmb from script to be made

gsmb = np.zeros((2,len(lat),len(lon)))
gsmb[0,:,:] = 2 * np.abs(fp_gr)
gsmb[1,:,:] = 3 * np.abs(fp_gr)
for s in [0,1]:
    gsmb[s,:,:][np.isnan(slr_md)]=np.nan

In [6]:
#Asmb from script to be made

asmb = np.zeros((2,len(lat),len(lon)))
asmb[0,:,:] = 2 * np.abs(fp_a)
asmb[1,:,:] = 3 * np.abs(fp_a)
for s in [0,1]:
    asmb[s,:,:][np.isnan(slr_md)]=np.nan

In [7]:
#LWS, GIA

gdyn = np.zeros((2,len(lat),len(lon)))
lws = np.zeros((2,len(lat),len(lon)))
gia = np.zeros((2,len(lat),len(lon)))
with xr.open_dataset('../rawdata/component-ts-slr-45.nc') as ds:
    gdyn_h = 100.*np.nanmean(ds['greendyn_h'].values[-20:,:,:],axis=0)
    gdyn_l = 100.*np.nanmean(ds['greendyn_l'].values[-20:,:,:],axis=0)
    lws_h  = 100.*np.nanmean(ds['grw_h'].values[-20:,:,:],axis=0)
    lws_l  = 100.*np.nanmean(ds['grw_l'].values[-20:,:,:],axis=0)
    for s in [0,1]:
        gia[s,:,:] = 100.*np.nanmean(ds['gia_sd'].values[-20:,:,:],axis=0)
        gia[s,:,:][np.isnan(slr_md)] = np.nan
        
gdyn[0,:,:] = np.abs(gdyn_h-gdyn_l)/(2*1.645)
lws[0,:,:] = np.abs(lws_h-lws_l)/(2*1.645)
lws[1,:,:] = np.abs(lws_h-lws_l)/(2*1.645)

with xr.open_dataset('../rawdata/component-ts-slr-85.nc') as ds:
    gdyn_h = 100.*np.nanmean(ds['greendyn_h'].values[-20:,:,:],axis=0)
    gdyn_l = 100.*np.nanmean(ds['greendyn_l'].values[-20:,:,:],axis=0)

gdyn[1,:,:] = np.abs(gdyn_h-gdyn_l)/(2*1.645)

/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: Mean of empty slice
  import sys
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: Mean of empty slice
/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: Mean of empty slice


In [8]:
#glac

mods = ['bcc-csm1-1','CanESM2','CCSM4','CNRM-CM5','CSIRO-Mk3-6-0','GFDL-CM3','GISS-E2-R','HadGEM2-ES','inmcm4','IPSL-CM5A-LR','MIROC-ESM','MIROC5','MRI-CGCM3','NorESM1-M']

glac = np.zeros((2,len(lat),len(lon)))

gglac = np.zeros((len(mods),len(lat),len(lon)))
for s,scen in enumerate(scens):
    for m,model in enumerate(mods):
        with xr.open_dataset(f'../data/rsl/{model}_{scen}.nc') as ds:
            gglac[m,:,:] = np.nanmean(ds['glac'][-20:,:,:].values,axis=0)  

    glac[s,:,:] = np.std(gglac,axis=0)       

/Users/erwin/miniconda2/envs/codep/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()


In [9]:
#save

#Save
glac2 = xr.DataArray(glac,dims=('scen','lat','lon'),coords={'scen':scens,'lat':lat,'lon':lon})
gsmb2 = xr.DataArray(gsmb,dims=('scen','lat','lon'),coords={'scen':scens,'lat':lat,'lon':lon})
gdyn2 = xr.DataArray(gdyn,dims=('scen','lat','lon'),coords={'scen':scens,'lat':lat,'lon':lon})
asmb2 = xr.DataArray(asmb,dims=('scen','lat','lon'),coords={'scen':scens,'lat':lat,'lon':lon})
adyn2 = xr.DataArray(adyn,dims=('scen','lat','lon'),coords={'scen':scens,'lat':lat,'lon':lon})
gia2  = xr.DataArray(gia,dims=('scen','lat','lon'),coords={'scen':scens,'lat':lat,'lon':lon})
lws2  = xr.DataArray(lws,dims=('scen','lat','lon'),coords={'scen':scens,'lat':lat,'lon':lon})
ds = xr.Dataset({'glac':glac2,'gsmb':gsmb2,'gdyn':gdyn2,'asmb':asmb2,'adyn':adyn2,'gia':gia2,'lws':lws2})
ds.to_netcdf(f'../data/sig_process.nc',)
ds.close()